In [1]:
import requests
import time
import os
from bs4 import BeautifulSoup
import itertools
import pandas
proxies = {'http': 'http://proxy.charite.de:8080',
           'https': 'https://proxy.charite.de:8080'}
print("Using HTTP proxy %s" % proxies['http'])

Using HTTP proxy http://proxy.charite.de:8080


In [2]:
# get the list
list_PIs = []
with open('PIs.txt') as fin:
    for line in fin:
        list_PIs.append(line)

In [4]:
# print one item of list
print(list_PIs[10])

# add a plus between name and last name
for ll in range(0,len(list_PIs)-1):
    list_PIs[ll] = list_PIs[ll].replace(' ','+')
    list_PIs[ll] = list_PIs[ll].replace('\n','')
    list_PIs[ll] = list_PIs[ll] + '[author]'


Nina+Romanczuk-Seiferth[author]


In [5]:
print(len(list_PIs))

40


In [6]:
# get combinations of authors
comb_list = list(itertools.combinations(list_PIs,2))
print(len(comb_list))

780


In [7]:
# make the query string
query_strings = []
for counter, ll in enumerate(comb_list):
    query_strings.append(ll[0] + '+AND+' + ll[1])
    
print(query_strings[30])


Christine+Heim[author][author]+AND+Christian+Beste[author][author]


In [8]:
# make th url data base query
url_queries = []

for counter, ll in enumerate(query_strings):
    url_queries.append('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=' + ll)
    
print(url_queries[30])



https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Christine+Heim[author][author]+AND+Christian+Beste[author][author]


In [9]:
# call all urls and get the number of results
all_counts = []
all_papers = []
for counter, curURL in enumerate(url_queries):
    s      = requests.session()
    r      = s.get(curURL,proxies=proxies)
    soup   = BeautifulSoup(r.text,"html5lib")
    
    # get the counts
    counts = soup.find_all("count")
    print(counter)
    if len(counts) == 0:
        print(soup)
        break
    all_counts.append(counts[0].get_text())
    time.sleep(1)
    
    # get the papers
    papers = soup.find_all("id")
    for counter, cur_paper in enumerate(papers):
        all_papers.append(papers[counter].get_text())
    

    

0
1
2
3


KeyboardInterrupt: 

In [45]:
# check result
#print(query_strings[0:5])
#print(all_counts)
#print(all_papers)

#print(comb_list[0])

# get the unique joint papers
unique_papers = set(all_papers)

print(unique_papers)

# print the number of joint publications
print(' ')
print('We have ' + str(len(unique_papers)) + ' joint publications.')
print('We have ' + str(len(all_papers)) + ' nonunique publications.')

{'18703319', '27374936', '25582269', '12645817', '29169660', '28798371', '22767373', '28671956', '30302403', '28740821', '30410047', '24756342', '27974817', '27909319', '30249471', '20673876', '25035082', '24470632', '29662068', '26433217', '30268001', '21790907', '27810934', '27086240', '29058369', '28491495', '27791105', '25566131', '24642287', '24060832', '28174031', '26982326', '30269148', '27415603', '25833034', '21309952', '28207073', '27079174', '26463560', '21262256', '26258487', '19581569', '17488322', '28649313', '12555235', '20570115', '23466521', '24254424', '29860087', '18606955', '23679679', '23564736', '22330316', '19490027', '27751654', '24283978', '25526597', '24176673', '27697987', '28589968', '26573969', '30583245', '24802075', '30111185', '24166646', '27694318', '29953935', '29656430', '26276438', '26900791', '27031567', '16899037', '14702246', '27145766', '27300740', '24079538', '25639749', '22520437', '27405334', '29050056', '24265700', '25913102', '25819024', '24

In [46]:
# define function for cleaning string
def clean_string(cur_str):
    cur_str = cur_str.replace('[author]','')
    cur_str = cur_str.replace('+',' ')
    return(cur_str)

# prepare columns for output file
first_author  = []
second_author = []

print(clean_string(comb_list[2][1]))
for counter, item in enumerate(comb_list):
    first_author.append(clean_string(comb_list[counter][0]))
    second_author.append(clean_string(comb_list[counter][1]))
    
    

Florian Schlagenhauf


In [47]:
# make a data frame ready for printing
df = pandas.DataFrame({'FirstAuthor': first_author, 'SecondAuthor': second_author,
                      'common_pubs': all_counts})
df

writer = pandas.ExcelWriter('wermitwem.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()
